In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, precision_score, accuracy_score, f1_score, recall_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip "/content/drive/MyDrive/Capstone/Features Extracted DataSets.zip" -d "/content/drive/MyDrive/Capstone/"

Reading Padel Data :

In [ ]:
bitter_train_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Padel/Cleaned Dataset/bitterTrainCleaned.csv")
bitter_test_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Padel/Cleaned Dataset/bitterTestCleaned.csv")
sweet_train_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Padel/Cleaned Dataset/sweet_train_clean.csv")
sweet_test_P = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Padel/Cleaned Dataset/sweet_test_clean.csv")

Reading Mordred Data :

In [ ]:
bitter_train_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Mordred/Cleaned Data/bitter_train_v5.csv")
bitter_test_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Mordred/Cleaned Data/bitter_test_v5.csv")
sweet_train_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Mordred/Cleaned Data/sweet_train_v5.csv")
sweet_test_M = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Mordred/Cleaned Data/sweet_test_v5.csv")

Reading Boruta Padel Data :

In [ ]:
bitter_train_PB = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Padel/padel_bitter_train_boruta.csv")
bitter_test_PB = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Padel/padel_bitter_test_boruta.csv")
sweet_train_PB = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Padel/sweet_train_boruta.csv")
sweet_test_PB = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Padel/sweet_test_boruta.csv")

Reading Boruta Modered Data :

In [ ]:
bitter_train_MB = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Modred/M_bitter_train_boruta.csv")
bitter_test_MB = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Modred/M_bitter_test_boruta.csv")
sweet_train_MB = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Modred/M_sweet_train_boruta.csv")
sweet_test_MB = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Boruta/Modred/M_sweet_test_boruta.csv")

Reading Combined Features :

In [ ]:
bitter_train_C = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Combined/combined_mordered_padel_bitter_train.csv")
bitter_test_C = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Combined/combined_mordered_padel_bitter_test.csv")
sweet_train_C = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Combined/combined_mordered_padel_sweet_train.csv")
sweet_test_C = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Combined/combined_mordered_padel_sweet_test.csv")

Script :

In [ ]:
def findPredictions(x_Train, x_Test, y_Train, y_Test, data_type):

  #Preparing data for PCA
  pca2 = PCA(n_components=2)
  x_pca2_data_train = pca2.fit_transform(x_Train)
  x_pca2_data_test = pca2.fit_transform(x_Test)

  pca10 = PCA(n_components=10)
  x_pca10_data_train = pca10.fit_transform(x_Train)
  x_pca10_data_test = pca10.fit_transform(x_Test)

  
  
  #Preparing data for Correlation matrix on features
  train_data = pd.concat([x_Train,y_Train],axis=1)
  test_data = pd.concat([x_Test,y_Test],axis=1)
  train_corr , test_corr = correlation_check(train_data, test_data,0.7)

  if(data_type == 'Sweet'):
    x_Train_corr = train_corr.drop(['Sweet'],axis=1)
    x_Test_corr = test_corr.drop(['Sweet'],axis=1)

  if(data_type == 'Bitter'):
    x_Train_corr = train_corr.drop(['Bitter'],axis=1)
    x_Test_corr = test_corr.drop(['Bitter'],axis=1)

  
  
  #Preparing data for Correlation matrix on features and Labels

  a = x_Train_corr.corr(method ='pearson').abs()
  df_temp = a.head(1)
  df_temp.fillna(0,inplace = True)

  colList = []
  for i in df_temp.columns:
    if(df_temp[i].item() < 0.1):
      colList.append(i)

  for i in colList:
    new_x_train_corr = x_Train_corr.drop(i,axis = 1)

  for i in colList:
    new_x_test_corr = x_Test_corr.drop(i,axis = 1)


  

  #Preparing data for selectkBest
  from sklearn.feature_selection import SelectKBest, f_classif
  
  X_new = SelectKBest(f_classif, k=200).fit(x_Train, y_Train)
  cols = X_new.get_support(indices=True)
  features_df_new_train = x_Train.iloc[:,cols]
  features_df_new_test = x_Test.iloc[:,cols]

  



  #Applying Models : 
  #---------------------

  #1. Logistic Regression :
  from sklearn.linear_model import LogisticRegression

  clfLR1 = LogisticRegression(random_state=0)
  
  print("Logistic Regression : ")
  print("------------------------")
  print()

  
  clfLR_Basic = clfLR1.fit(x_Train, y_Train)
  y_pred_Basic = clfLR_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic)," Recall: ",recall_score(y_Test, y_pred_Basic),"  F1: ",f1_score(y_Test, y_pred_Basic, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfLR2 = LogisticRegression(random_state=0)

  clfLRpca2 = clfLR2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfLRpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2)," Recall: ",recall_score(y_Test, y_predPca2),"  F1: ",f1_score(y_Test, y_predPca2, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfLR3 = LogisticRegression(random_state=0)

  clfLRpca10 = clfLR3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfLRpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10)," Recall: ",recall_score(y_Test, y_predPca10),"  F1: ",f1_score(y_Test, y_predPca10, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  clfLR4 = LogisticRegression(random_state=0)

  clfLR_CMF = clfLR4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfLR_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF)," Recall: ",recall_score(y_Test, y_pred_CMF),"  F1: ",f1_score(y_Test, y_pred_CMF, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfLR5 = LogisticRegression(random_state=0)

  clfLR_CMFL = clfLR5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL)," Recall: ",recall_score(y_Test, y_pred_CMFL),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfLR6 = LogisticRegression(random_state=0)

  clfLR_selectK = clfLR6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfLR_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK)," Recall: ",recall_score(y_Test, y_pred_selectK),"  F1: ",f1_score(y_Test, y_pred_selectK, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()

  #=====================================================================================================================================================

  #2. Random forgest :
  from sklearn.ensemble import RandomForestClassifier

  clfRF1 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)
  
  print("Random Forest : ")
  print("----------------")
  print()

  clfRF_Basic = clfRF1.fit(x_Train, y_Train)
  y_pred_Basic = clfRF_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic)," Recall: ",recall_score(y_Test, y_pred_Basic),"  F1: ",f1_score(y_Test, y_pred_Basic, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfRF2 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)


  clfRFpca2 = clfRF2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfRFpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2)," Recall: ",recall_score(y_Test, y_predPca2),"  F1: ",f1_score(y_Test, y_predPca2, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfRF3 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)


  clfRFpca10 = clfRF3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfRFpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10)," Recall: ",recall_score(y_Test, y_predPca10),"  F1: ",f1_score(y_Test, y_predPca10, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  clfRF4 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)

  clfRF_CMF = clfRF4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfRF_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF)," Recall: ",recall_score(y_Test, y_pred_CMF),"  F1: ",f1_score(y_Test, y_pred_CMF, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfRF5 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)

  clfLR_CMFL = clfRF5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL)," Recall: ",recall_score(y_Test, y_pred_CMFL),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()


  clfRF6 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=10, criterion='entropy', random_state=30)

  clfRF_selectK = clfRF6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfRF_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK)," Recall: ",recall_score(y_Test, y_pred_selectK),"  F1: ",f1_score(y_Test, y_pred_selectK, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()


  


  #3. Adaboost
  from sklearn.ensemble import AdaBoostClassifier

  clfAB1 = AdaBoostClassifier(n_estimators=100, random_state=0)

  print("AdaBoost : ")
  print("----------------")
  print()

  clfAB_Basic = clfAB1.fit(x_Train, y_Train)
  y_pred_Basic = clfAB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic)," Recall: ",recall_score(y_Test, y_pred_Basic),"  F1: ",f1_score(y_Test, y_pred_Basic, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfAB2 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca2 = clfAB2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfABpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2)," Recall: ",recall_score(y_Test, y_predPca2),"  F1: ",f1_score(y_Test, y_predPca2, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfAB3 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca10 = clfAB3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfABpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10)," Recall: ",recall_score(y_Test, y_predPca10),"  F1: ",f1_score(y_Test, y_predPca10, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  clfAB4 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMF = clfAB4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfAB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF)," Recall: ",recall_score(y_Test, y_pred_CMF),"  F1: ",f1_score(y_Test, y_pred_CMF, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfAB5 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMFL = clfAB5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfAB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL)," Recall: ",recall_score(y_Test, y_pred_CMFL),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfAB6 = AdaBoostClassifier(n_estimators=100, random_state=0)


  clfAB_selectK = clfAB6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfAB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK)," Recall: ",recall_score(y_Test, y_pred_selectK),"  F1: ",f1_score(y_Test, y_pred_selectK, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()


  
  #4. XGBoostClassifier
  import xgboost as xg

  xgb_r1 = xg.XGBClassifier(max_depth=10)

  print("XGBoostClassifier : ")
  print("----------------")
  print()

  clfXGB_Basic = xgb_r1.fit(x_Train, y_Train)
  y_pred_Basic = clfXGB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic)," Recall: ",recall_score(y_Test, y_pred_Basic),"  F1: ",f1_score(y_Test, y_pred_Basic, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  xgb_r2 = xg.XGBClassifier(max_depth=10)

  clfXGBpca2 = xgb_r2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfXGBpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2)," Recall: ",recall_score(y_Test, y_predPca2),"  F1: ",f1_score(y_Test, y_predPca2, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  xgb_r3 = xg.XGBClassifier(max_depth=10)


  clfXGBpca10 = xgb_r3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfXGBpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10)," Recall: ",recall_score(y_Test, y_predPca10),"  F1: ",f1_score(y_Test, y_predPca10, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  xgb_r4 = xg.XGBClassifier(max_depth=10)

  clfXGB_CMF = xgb_r4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfXGB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF)," Recall: ",recall_score(y_Test, y_pred_CMF),"  F1: ",f1_score(y_Test, y_pred_CMF, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  xgb_r5 = xg.XGBClassifier(max_depth=10)

  clfXGB_CMFL = xgb_r5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfXGB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL)," Recall: ",recall_score(y_Test, y_pred_CMFL),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  xgb_r6 = xg.XGBClassifier(max_depth=10)

  clfXGB_selectK = xgb_r6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfXGB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK)," Recall: ",recall_score(y_Test, y_pred_selectK),"  F1: ",f1_score(y_Test, y_pred_selectK, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()

  #4. LGBM Classifier
  import lightgbm as lgb
  from lightgbm import LGBMClassifier

  lgb_r1 = LGBMClassifier(max_depth=10)

  print("LGBMClassifier : ")
  print("----------------")
  print()

  clfLGBM_Basic = lgb_r1.fit(x_Train, y_Train)
  y_pred_Basic = clfLGBM_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic)," Recall: ",recall_score(y_Test, y_pred_Basic),"  F1: ",f1_score(y_Test, y_pred_Basic, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  lgb_r2 = LGBMClassifier(max_depth=10)

  clfLGBMpca2 = lgb_r2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfLGBMpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2)," Recall: ",recall_score(y_Test, y_predPca2),"  F1: ",f1_score(y_Test, y_predPca2, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  lgb_r3 = LGBMClassifier(max_depth=10)


  clfLGBMpca10 = lgb_r3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfLGBMpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10)," Recall: ",recall_score(y_Test, y_predPca10),"  F1: ",f1_score(y_Test, y_predPca10, average='binary')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  lgb_r4 = LGBMClassifier(max_depth=10)

  clfLGBM_CMF = lgb_r4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfLGBM_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF)," Recall: ",recall_score(y_Test, y_pred_CMF),"  F1: ",f1_score(y_Test, y_pred_CMF, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  lgb_r5 = LGBMClassifier(max_depth=10)

  clfLGBM_CMFL = lgb_r5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLGBM_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL)," Recall: ",recall_score(y_Test, y_pred_CMFL),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  lgb_r6 = LGBMClassifier(max_depth=10)

  clfLGBM_selectK = lgb_r6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfLGBM_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK)," Recall: ",recall_score(y_Test, y_pred_selectK),"  F1: ",f1_score(y_Test, y_pred_selectK, average='binary')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()  
  

Supporting Functions :

In [ ]:
def correlation_check(traindata,testdata,thresh): # drop columns above certain threshold    
        corr_matrix = traindata.corr()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        to_drop = [column for column in upper.columns if any(upper[column] >thresh)]
        trainset=traindata.drop(traindata[to_drop], axis=1)
        testset=testdata.drop(testdata[to_drop],axis=1)
        return trainset,testset

In [ ]:
def gridSearchOnLogReg(x, y):
  grid={
        "C":np.logspace(-3,3,7,1,0),
        "penalty":["l1","l2","elasticnet","none"],
        "max_iter":[50,80,100],
        "solver":["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
        }
  logreg=LogisticRegression()
  logreg_cv=GridSearchCV(logreg,grid)
  logreg_cv.fit(x,y)
  return logreg_cv

RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=8, criterion='entropy', random_state=30)

## **Padel :**

---



Sweet

In [ ]:
y_train = sweet_train_P['Sweet']
x_train = sweet_train_P.drop(['Sweet','Name','Name.1','Taste','Unnamed: 0'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = sweet_test_P['Sweet']
x_test = sweet_test_P.drop(['Sweet','Name','Name.1','Taste','Unnamed: 0'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Sweet")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.8382352941176471  Recall:  0.5277777777777778   F1:  0.6477272727272728  Accuracy:  0.6149068322981367

PCA with n = 2 ==>                 Precision:  0.6883116883116883  Recall:  0.9814814814814815   F1:  0.8091603053435115  Accuracy:  0.6894409937888198

PCA with n = 10 ==>                Precision:  0.6594202898550725  Recall:  0.8425925925925926   F1:  0.7398373983739838  Accuracy:  0.6024844720496895

Correlation Matrix on Features ==> Precision:  0.7363636363636363  Recall:  0.75   F1:  0.7431192660550459  Accuracy:  0.6521739130434783

CorMatrix on Features & Labels ==> Precision:  0.7297297297297297  Recall:  0.75   F1:  0.7397260273972601  Accuracy:  0.6459627329192547

SelectK best with k = 200 ==>      Precision:  0.92  Recall:  0.6388888888888888   F1:  0.7540983606557377  Accuracy:  0.7204968944099379

Random Forest : 
----------------

Basic ==>                          Preci

Bitter

In [ ]:
y_train = bitter_train_P['Bitter']
x_train = bitter_train_P.drop(['Bitter','Name','Name.1','Taste'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = bitter_test_P['Bitter']
x_test = bitter_test_P.drop(['Bitter','Name','Name.1','Taste'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Bitter")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.7407407407407407  Recall:  0.5714285714285714   F1:  0.6451612903225806  Accuracy:  0.6140350877192983

PCA with n = 2 ==>                 Precision:  0.14285714285714285  Recall:  0.009523809523809525   F1:  0.01785714285714286  Accuracy:  0.3567251461988304

PCA with n = 10 ==>                Precision:  0.375  Recall:  0.05714285714285714   F1:  0.09917355371900827  Accuracy:  0.36257309941520466

Correlation Matrix on Features ==> Precision:  0.8035714285714286  Recall:  0.42857142857142855   F1:  0.5590062111801243  Accuracy:  0.5847953216374269

CorMatrix on Features & Labels ==> Precision:  0.8  Recall:  0.41904761904761906   F1:  0.55  Accuracy:  0.5789473684210527

SelectK best with k = 200 ==>      Precision:  0.7083333333333334  Recall:  0.4857142857142857   F1:  0.576271186440678  Accuracy:  0.5614035087719298

Random Forest : 
----------------

Basic ==>                       

## **Mordred**

---



Sweet

In [ ]:
y_train = sweet_train_M['Sweet']
x_train = sweet_train_M.drop(['Sweet','Name','Taste'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = sweet_test_M['Sweet']
x_test = sweet_test_M.drop(['Sweet','Name','Taste'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Sweet")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

PCA with n = 2 ==>                 Precision:  0.6644295302013423  Recall:  0.9611650485436893   F1:  0.7857142857142857  Accuracy:  0.6470588235294118

PCA with n = 10 ==>                Precision:  0.6644295302013423  Recall:  0.9611650485436893   F1:  0.7857142857142857  Accuracy:  0.6470588235294118

Correlation Matrix on Features ==> Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

CorMatrix on Features & Labels ==> Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

SelectK best with k = 200 ==>      Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

Random Forest : 
----------------

Basic ==>                          Precision:  0.8589743589743589  Recall:  0.6504854368932039   F1:  0.7403314917127072  Accuracy:  0.6928104575163399

PCA with n = 2 ==>   

Bitter

In [ ]:
y_train = bitter_train_M['Bitter']
x_train = bitter_train_M.drop(['Bitter','Name','Taste'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = bitter_test_M['Bitter']
x_test = bitter_test_M.drop(['Bitter','Name','Taste'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Bitter")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  1.0  Recall:  0.009523809523809525   F1:  0.01886792452830189  Accuracy:  0.391812865497076

PCA with n = 2 ==>                 Precision:  0.4864864864864865  Recall:  0.5142857142857142   F1:  0.5  Accuracy:  0.3684210526315789

PCA with n = 10 ==>                Precision:  0.5877192982456141  Recall:  0.638095238095238   F1:  0.6118721461187215  Accuracy:  0.5029239766081871

Correlation Matrix on Features ==> Precision:  0.6140350877192983  Recall:  1.0   F1:  0.7608695652173912  Accuracy:  0.6140350877192983

CorMatrix on Features & Labels ==> Precision:  0.6140350877192983  Recall:  1.0   F1:  0.7608695652173912  Accuracy:  0.6140350877192983

SelectK best with k = 200 ==>      Precision:  0.6140350877192983  Recall:  1.0   F1:  0.7608695652173912  Accuracy:  0.6140350877192983

Random Forest : 
----------------

Basic ==>                          Precision:  0.8404255319148937  Recal

## **Boruta Padel**

---



Sweet

In [ ]:
y_train = sweet_train_PB['Sweet']
x_train = sweet_train_PB.drop(['Sweet'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = sweet_test_PB['Sweet']
x_test = sweet_test_PB.drop(['Sweet'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Sweet")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.8354430379746836  Recall:  0.6111111111111112   F1:  0.7058823529411765  Accuracy:  0.6583850931677019

PCA with n = 2 ==>                 Precision:  0.6835443037974683  Recall:  1.0   F1:  0.8120300751879699  Accuracy:  0.6894409937888198

PCA with n = 10 ==>                Precision:  0.7875  Recall:  0.5833333333333334   F1:  0.6702127659574468  Accuracy:  0.6149068322981367

Correlation Matrix on Features ==> Precision:  0.71875  Recall:  0.6388888888888888   F1:  0.676470588235294  Accuracy:  0.5900621118012422

CorMatrix on Features & Labels ==> Precision:  0.71875  Recall:  0.6388888888888888   F1:  0.676470588235294  Accuracy:  0.5900621118012422

SelectK best with k = 200 ==>      Precision:  0.8507462686567164  Recall:  0.5277777777777778   F1:  0.6514285714285716  Accuracy:  0.6211180124223602

Random Forest : 
----------------

Basic ==>                          Precision:  0.

Bitter

In [ ]:
y_train = bitter_train_PB['Bitter']
x_train = bitter_train_PB.drop(['Bitter'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = bitter_test_PB['Bitter']
x_test = bitter_test_PB.drop(['Bitter'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Bitter")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.7848101265822784  Recall:  0.5904761904761905   F1:  0.6739130434782609  Accuracy:  0.6491228070175439

PCA with n = 2 ==>                 Precision:  0.8928571428571429  Recall:  0.47619047619047616   F1:  0.6211180124223602  Accuracy:  0.6432748538011696

PCA with n = 10 ==>                Precision:  0.6973684210526315  Recall:  0.5047619047619047   F1:  0.585635359116022  Accuracy:  0.5614035087719298

Correlation Matrix on Features ==> Precision:  0.8979591836734694  Recall:  0.41904761904761906   F1:  0.5714285714285715  Accuracy:  0.6140350877192983

CorMatrix on Features & Labels ==> Precision:  0.9019607843137255  Recall:  0.4380952380952381   F1:  0.5897435897435896  Accuracy:  0.6257309941520468

SelectK best with k = 200 ==>      Precision:  0.7605633802816901  Recall:  0.5142857142857142   F1:  0.6136363636363636  Accuracy:  0.6023391812865497

Random Forest : 
---------------

## **Boruta Mordered**

---



Sweet

In [ ]:
y_train = sweet_train_MB['Sweet']
x_train = sweet_train_MB.drop(['Sweet'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = sweet_test_MB['Sweet']
x_test = sweet_test_MB.drop(['Sweet'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Sweet")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.8088235294117647  Recall:  0.5339805825242718   F1:  0.6432748538011697  Accuracy:  0.6013071895424836

PCA with n = 2 ==>                 Precision:  0.6545454545454545  Recall:  0.6990291262135923   F1:  0.6760563380281691  Accuracy:  0.5490196078431373

PCA with n = 10 ==>                Precision:  0.5853658536585366  Recall:  0.46601941747572817   F1:  0.518918918918919  Accuracy:  0.41830065359477125

Correlation Matrix on Features ==> Precision:  0.8051948051948052  Recall:  0.6019417475728155   F1:  0.6888888888888889  Accuracy:  0.6339869281045751

CorMatrix on Features & Labels ==> Precision:  0.7948717948717948  Recall:  0.6019417475728155   F1:  0.6850828729281767  Accuracy:  0.6274509803921569

SelectK best with k = 200 ==>      Precision:  0.835820895522388  Recall:  0.5436893203883495   F1:  0.6588235294117647  Accuracy:  0.6209150326797386

Random Forest : 
----------------

Bitter

In [ ]:
y_train = bitter_train_MB['Bitter']
x_train = bitter_train_MB.drop(['Bitter'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = bitter_test_MB['Bitter']
x_test = bitter_test_MB.drop(['Bitter'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Bitter")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.6071428571428571  Recall:  0.9714285714285714   F1:  0.7472527472527473  Accuracy:  0.5964912280701754

PCA with n = 2 ==>                 Precision:  0.4838709677419355  Recall:  0.42857142857142855   F1:  0.4545454545454545  Accuracy:  0.3684210526315789

PCA with n = 10 ==>                Precision:  0.4838709677419355  Recall:  0.42857142857142855   F1:  0.4545454545454545  Accuracy:  0.3684210526315789

Correlation Matrix on Features ==> Precision:  0.6140350877192983  Recall:  1.0   F1:  0.7608695652173912  Accuracy:  0.6140350877192983

CorMatrix on Features & Labels ==> Precision:  0.6140350877192983  Recall:  1.0   F1:  0.7608695652173912  Accuracy:  0.6140350877192983

SelectK best with k = 200 ==>      Precision:  0.6071428571428571  Recall:  0.9714285714285714   F1:  0.7472527472527473  Accuracy:  0.5964912280701754

Random Forest : 
----------------

Basic ==>                 

## **Combined**

---



In [ ]:
sweet_train_C

,Unnamed: 0,Name,Taste,Sweet,Name.1,nAcid_x,ALogP,ALogp2,AMR,apol_x,...,SRW10_y,TSRW10,MW_y,AMW_y,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,Sucrose,Sweet,True,AUTOGEN_molecule_1,0,-4.3105,18.580410,68.7741,44.611446,...,10.247042,72.317821,342.116212,7.602582,1110,43,120.0,147.0,10.451389,5.291667
1,1,"Sucralose / 4,1',6'-Trichloro-galactosucrose",Sweet,True,AUTOGEN_molecule_2,0,-0.6967,0.485391,77.9271,46.745067,...,10.247042,72.317821,396.014551,9.428918,1110,43,120.0,147.0,10.451389,5.291667
2,2,Aspartame/Aspartyl-phenylalanine methylester,Sweet,True,AUTOGEN_molecule_3,1,-1.8303,3.349998,46.9315,42.852274,...,9.493487,53.670480,294.121572,7.541579,1000,27,96.0,105.0,8.916667,4.888889
3,3,Tagatose,Sweet,True,AUTOGEN_molecule_4,0,-2.5636,6.572045,36.3608,23.373516,...,9.569063,43.370989,180.063388,7.502641,178,20,60.0,71.0,6.145833,2.722222
4,4,Isomaltulose/Palatinose,Sweet,True,AUTOGEN_molecule_5,0,-4.3105,18.580410,68.7741,44.611446,...,10.192007,72.204129,342.116212,7.602582,1220,41,120.0,145.0,10.451389,5.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,2176,2-(2-Ethoxyethyl)pyridine,Bitter,False,AUTOGEN_molecule_613,0,-0.0778,0.006053,19.2334,26.410309,...,8.606851,37.313444,123.068414,6.837134,88,9,40.0,43.0,3.472222,2.166667
2168,2177,4-BENZYLPYRIDINE,Bitter,False,AUTOGEN_molecule_614,0,0.4562,0.208118,4.6010,29.554723,...,8.590258,37.289972,124.052429,7.297202,90,9,40.0,43.0,3.472222,2.166667
2169,2178,TRIDECANAL,Bitter,False,AUTOGEN_molecule_615,0,-2.9863,8.917988,45.8578,41.018618,...,9.282754,45.217583,183.068414,7.959496,307,18,68.0,77.0,3.833333,3.222222
2170,2179,L-phenylalanine,Bitter,False,AUTOGEN_molecule_616,1,-1.0844,1.175923,19.6296,25.878723,...,8.645235,53.584828,152.083730,6.612336,173,9,50.0,53.0,4.583333,2.500000


Sweet

In [ ]:
y_train = sweet_train_C['Sweet']
x_train = sweet_train_C.drop(['Sweet','Name','Taste','Name.1'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = sweet_test_C['Sweet']
x_test = sweet_test_C.drop(['Sweet','Name','Taste','Name.1'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Sweet")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  1.0  Recall:  0.009708737864077669   F1:  0.01923076923076923  Accuracy:  0.3333333333333333

PCA with n = 2 ==>                 Precision:  0.6644295302013423  Recall:  0.9611650485436893   F1:  0.7857142857142857  Accuracy:  0.6470588235294118

PCA with n = 10 ==>                Precision:  0.6764705882352942  Recall:  0.8932038834951457   F1:  0.7698744769874478  Accuracy:  0.6405228758169934

Correlation Matrix on Features ==> Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

CorMatrix on Features & Labels ==> Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.32679738562091504

SelectK best with k = 200 ==>      Precision:  0.743801652892562  Recall:  0.8737864077669902   F1:  0.8035714285714285  Accuracy:  0.7124183006535948

Random Forest : 
----------------

Basic ==>                          Precision:  0.8295454545454546  Recall:  0.7087378640776699   F1

Bitter

In [ ]:
y_train = bitter_train_C['Bitter']
x_train = bitter_train_C.drop(['Bitter','Name','Taste','Name.1'],axis=1)
y_train.replace({True:1, False:0},inplace=True)
y_test = bitter_test_C['Bitter']
x_test = bitter_test_C.drop(['Bitter','Name','Taste','Name.1'],axis=1)
y_test.replace({True:1, False:0},inplace=True)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

In [ ]:
findPredictions(x_train, x_test, y_train, y_test, "Bitter")

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.7256637168141593  Recall:  0.9534883720930233   F1:  0.8241206030150754  Accuracy:  0.7058823529411765

PCA with n = 2 ==>                 Precision:  0.0  Recall:  0.0   F1:  0.0  Accuracy:  0.2605042016806723

PCA with n = 10 ==>                Precision:  0.6153846153846154  Recall:  0.46511627906976744   F1:  0.5298013245033113  Accuracy:  0.40336134453781514

Correlation Matrix on Features ==> Precision:  0.7589285714285714  Recall:  0.9883720930232558   F1:  0.8585858585858586  Accuracy:  0.7647058823529411

CorMatrix on Features & Labels ==> Precision:  0.7589285714285714  Recall:  0.9883720930232558   F1:  0.8585858585858586  Accuracy:  0.7647058823529411

SelectK best with k = 200 ==>      Precision:  0.7226890756302521  Recall:  1.0   F1:  0.8390243902439024  Accuracy:  0.7226890756302521

Random Forest : 
----------------

Basic ==>                          Precision:  0.95  Rec

**CHEMTASTEDB**

In [ ]:
def findPredictionsChemTasteDB(x_Train, x_Test, y_Train, y_Test):

  #Preparing data for PCA
  pca2 = PCA(n_components=2)
  x_pca2_data_train = pca2.fit_transform(x_Train)
  x_pca2_data_test = pca2.fit_transform(x_Test)

  pca10 = PCA(n_components=10)
  x_pca10_data_train = pca10.fit_transform(x_Train)
  x_pca10_data_test = pca10.fit_transform(x_Test)

  
  
  #Preparing data for Correlation matrix on features
  train_data = pd.concat([x_Train,y_Train],axis=1)
  test_data = pd.concat([x_Test,y_Test],axis=1)
  train_corr , test_corr = correlation_check(train_data, test_data,0.8)

  
  x_Train_corr = train_corr.drop(['Class taste'],axis=1)
  x_Test_corr = test_corr.drop(['Class taste'],axis=1)

  
  
  #Preparing data for Correlation matrix on features and Labels

  a = x_Train_corr.corr(method ='pearson').abs()
  df_temp = a.head(1)
  df_temp.fillna(0,inplace = True)

  colList = []
  for i in df_temp.columns:
    if(df_temp[i].item() < 0.1):
      colList.append(i)

  for i in colList:
    new_x_train_corr = x_Train_corr.drop(i,axis = 1)

  for i in colList:
    new_x_test_corr = x_Test_corr.drop(i,axis = 1)


  

  #Preparing data for selectkBest
  from sklearn.feature_selection import SelectKBest, f_classif
  
  X_new = SelectKBest(f_classif, k=200).fit(x_Train, y_Train)
  cols = X_new.get_support(indices=True)
  features_df_new_train = x_Train.iloc[:,cols]
  features_df_new_test = x_Test.iloc[:,cols]

  



  #Applying Models : 
  #---------------------

  #1. Logistic Regression :
  from sklearn.linear_model import LogisticRegression

  clfLR1 = LogisticRegression(random_state=0)
  
  print("Logistic Regression : ")
  print("------------------------")
  print()

  
  clfLR_Basic = clfLR1.fit(x_Train, y_Train)
  y_pred_Basic = clfLR_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfLR2 = LogisticRegression(random_state=0)

  clfLRpca2 = clfLR2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfLRpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfLR3 = LogisticRegression(random_state=0)

  clfLRpca10 = clfLR3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfLRpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()

  clfLR4 = LogisticRegression(random_state=0)

  clfLR_CMF = clfLR4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfLR_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfLR5 = LogisticRegression(random_state=0)

  clfLR_CMFL = clfLR5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfLR6 = LogisticRegression(random_state=0)

  clfLR_selectK = clfLR6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfLR_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()

  #=====================================================================================================================================================

  #2. Random forgest :
  from sklearn.ensemble import RandomForestClassifier

  clfRF1 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)
  
  print("Random Forest : ")
  print("----------------")
  print()

  clfRF_Basic = clfRF1.fit(x_Train, y_Train)
  y_pred_Basic = clfRF_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfRF2 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)


  clfRFpca2 = clfRF2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfRFpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfRF3 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)


  clfRFpca10 = clfRF3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfRFpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()


  clfRF4 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)

  clfRF_CMF = clfRF4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfRF_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfRF5 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)

  clfLR_CMFL = clfRF5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfLR_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()


  clfRF6 = RandomForestClassifier(max_features='auto', n_estimators= 200, max_depth=20, criterion='entropy', random_state=30)

  clfRF_selectK = clfRF6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfRF_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()



  


  #3. Adaboost
  from sklearn.ensemble import AdaBoostClassifier

  clfAB1 = AdaBoostClassifier(n_estimators=100, random_state=0)

  print("AdaBoost : ")
  print("----------------")
  print()

  clfAB_Basic = clfAB1.fit(x_Train, y_Train)
  y_pred_Basic = clfAB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  clfAB2 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca2 = clfAB2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfABpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  clfAB3 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfABpca10 = clfAB3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfABpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()


  clfAB4 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMF = clfAB4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfAB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  clfAB5 = AdaBoostClassifier(n_estimators=100, random_state=0)

  clfAB_CMFL = clfAB5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfAB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  clfAB6 = AdaBoostClassifier(n_estimators=100, random_state=0)


  clfAB_selectK = clfAB6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfAB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()



  
  #4. XGBoostClassifier
  import xgboost as xg

  xgb_r1 = xg.XGBClassifier(max_depth=10)

  print("XGBoostClassifier : ")
  print("----------------")
  print()

  clfXGB_Basic = xgb_r1.fit(x_Train, y_Train)
  y_pred_Basic = clfXGB_Basic.predict(x_Test)
  print("Basic ==>                          Precision: ",precision_score(y_Test, y_pred_Basic,average='weighted')," Recall: ",recall_score(y_Test, y_pred_Basic,average='weighted'),"  F1: ",f1_score(y_Test, y_pred_Basic, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_Basic))
  print()

  xgb_r2 = xg.XGBClassifier(max_depth=20)

  clfXGBpca2 = xgb_r2.fit(x_pca2_data_train, y_Train)
  y_predPca2 = clfXGBpca2.predict(x_pca2_data_test)
  print("PCA with n = 2 ==>                 Precision: ",precision_score(y_Test, y_predPca2, average='weighted')," Recall: ",recall_score(y_Test, y_predPca2, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca2, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_predPca2))
  print()

  xgb_r3 = xg.XGBClassifier(max_depth=20)


  clfXGBpca10 = xgb_r3.fit(x_pca10_data_train, y_Train)
  y_predPca10 = clfXGBpca10.predict(x_pca10_data_test)
  print("PCA with n = 10 ==>                Precision: ",precision_score(y_Test, y_predPca10, average='weighted')," Recall: ",recall_score(y_Test, y_predPca10, average='weighted'),"  F1: ",f1_score(y_Test, y_predPca10, average='macro')," Accuracy: ",accuracy_score(y_Test, y_predPca10))
  print()


  xgb_r4 = xg.XGBClassifier(max_depth=20)

  clfXGB_CMF = xgb_r4.fit(x_Train_corr, y_Train)
  y_pred_CMF = clfXGB_CMF.predict(x_Test_corr)
  print("Correlation Matrix on Features ==> Precision: ",precision_score(y_Test, y_pred_CMF, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMF, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMF, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMF))
  print()

  xgb_r5 = xg.XGBClassifier(max_depth=20)

  clfXGB_CMFL = xgb_r5.fit(new_x_train_corr, y_Train)
  y_pred_CMFL = clfXGB_CMFL.predict(new_x_test_corr)
  print("CorMatrix on Features & Labels ==> Precision: ",precision_score(y_Test, y_pred_CMFL, average='weighted')," Recall: ",recall_score(y_Test, y_pred_CMFL, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_CMFL, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_CMFL))
  print()

  xgb_r6 = xg.XGBClassifier(max_depth=20)

  clfXGB_selectK = xgb_r6.fit(features_df_new_train, y_Train)
  y_pred_selectK = clfXGB_selectK.predict(features_df_new_test)
  print("SelectK best with k = 200 ==>      Precision: ",precision_score(y_Test, y_pred_selectK, average='weighted')," Recall: ",recall_score(y_Test, y_pred_selectK, average='weighted'),"  F1: ",f1_score(y_Test, y_pred_selectK, average='weighted')," Accuracy: ",accuracy_score(y_Test, y_pred_selectK))
  print()


  
  

In [ ]:
def correlation_check(traindata,testdata,thresh): # drop columns above certain threshold    
        corr_matrix = traindata.corr()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        to_drop = [column for column in upper.columns if any(upper[column] >thresh)]
        trainset=traindata.drop(traindata[to_drop], axis=1)
        testset=testdata.drop(testdata[to_drop],axis=1)
        return trainset,testset

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
def gridSearchOnLogReg(x, y):
  grid={
        "C":np.logspace(-3,3,7,1,0),
        "penalty":["l1","l2","elasticnet","none"],
        "max_iter":[50,80,100],
        "solver":["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
        }
  logreg=LogisticRegression()
  logreg_cv=GridSearchCV(logreg,grid)
  logreg_cv.fit(x,y)
  return logreg_cv

In [ ]:
chemTasteDB = pd.read_csv("/content/drive/MyDrive/Capstone/Features Extracted DataSets/Combined/combined_features_chemTaste.csv")

In [ ]:
chemTasteDB = chemTasteDB.dropna()

In [ ]:
chemTasteDB.isnull().values.any()

False

In [ ]:
y = chemTasteDB['Class taste']
x = chemTasteDB.drop(['Class taste','Name','Taste'],axis=1)
y=y.to_frame()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
findPredictionsChemTasteDB(x_train, x_test, y_train, y_test)

Logistic Regression : 
------------------------

Basic ==>                          Precision:  0.3872237348045693  Recall:  0.4475920679886686   F1:  0.35010916736662057  Accuracy:  0.4475920679886686

PCA with n = 2 ==>                 Precision:  0.25274639304301993  Recall:  0.358356940509915   F1:  0.2600992531831407  Accuracy:  0.358356940509915

PCA with n = 10 ==>                Precision:  0.38133625011197814  Recall:  0.37960339943342775   F1:  0.2852477411315442  Accuracy:  0.37960339943342775

Correlation Matrix on Features ==> Precision:  0.33529567988668557  Recall:  0.4178470254957507   F1:  0.3044957626101458  Accuracy:  0.4178470254957507

CorMatrix on Features & Labels ==> Precision:  0.3503118085727973  Recall:  0.4164305949008499   F1:  0.2709276163163478  Accuracy:  0.4164305949008499

SelectK best with k = 200 ==>      Precision:  0.16872778049739584  Recall:  0.41076487252124644   F1:  0.23920042777341657  Accuracy:  0.41076487252124644

Random Forest : 
--------